In [16]:
# Cell 1: Import thư viện
import pandas as pd
import math
from collections import defaultdict
from underthesea import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


In [17]:
# Cell 2: Đọc CSV
# CSV có cột: id, subject, body, label
# Lưu ý: dùng encoding='utf-8' để đọc tiếng Việt
df = pd.read_csv("Train.csv", encoding='utf-8')

# Gộp subject + body thành text
df['text'] = df['subject'].fillna('') + " " + df['body'].fillna('')

print("5 dòng đầu của dataset:")
print(df[['id', 'subject', 'body', 'label']].head())


5 dòng đầu của dataset:
   id                                      subject  \
0   1      Thông báo vi phạm và yêu cầu thanh toán   
1   2                       Thông báo trúng thưởng   
2   3                       Đơn của bạn bị từ chối   
3   4  Đảm bảo giảm 10–12 cân trong 30 ngày 10.162   
4   5                        Cơ hội việc làm ở nhà   

                                                body label  
0  Xin chào, Bạn đang vi phạm Khoản 1 Điều 466 Bộ...  spam  
1  Xin chúc mừng bạn dxdiag@gmail.com đã trúng th...  spam  
2  Bạn đang ngập trong nợ nần?\nĐây là những gì c...  spam  
3  Đánh bại nguy cơ ung thư!\nhttp://www.adclick....  spam  
4  Một cơ hội kinh doanh tại nhà đang gõ cửa nhà ...  spam  


In [18]:
# Cell 3: Tiền xử lý text
def preprocess(text):
    """Xóa xuống dòng, nối thành câu, thêm dấu '.' cuối"""
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip()]
    return ". ".join(lines) + "." if lines else ""


In [19]:
# Cell 4: Tokenize tiếng Việt
def tokenize(text):
    return word_tokenize(preprocess(text.lower()), format='text').split()


# Test tokenize
sample_text = "Nhận quà tặng miễn phí, click để nhận"
print("Tokenize ví dụ:", tokenize(sample_text))


Tokenize ví dụ: ['nhận', 'quà', 'tặng', 'miễn_phí', ',', 'click', 'để', 'nhận', '.']


In [20]:
# Cell 5: Tạo bộ đếm Naive Bayes
class_counts = defaultdict(int)
word_counts = defaultdict(lambda: defaultdict(int))
total_words = defaultdict(int)

for text, label in zip(df['text'], df['label']):
    class_counts[label] += 1
    words = tokenize(text)
    for word in words:
        word_counts[label][word] += 1
        total_words[label] += 1

print("Số email theo lớp:", dict(class_counts))
print("Tổng số từ mỗi lớp:", dict(total_words))
print("Một phần word_counts cho spam:", dict(list(word_counts['spam'].items())[:5]))


Số email theo lớp: {'spam': 150, 'ham': 100}
Tổng số từ mỗi lớp: {'spam': 14391, 'ham': 12239}
Một phần word_counts cho spam: {'thông_báo': 42, 'vi_phạm': 5, 'và': 185, 'yêu_cầu': 13, 'thanh_toán': 20}


In [21]:
# Cell 6: Hàm dự đoán xác suất
def predict_proba(email, verbose=False):
    words = tokenize(email)
    scores = {}
    vocab = set()
    for label in class_counts:
        for word in word_counts[label]:
            vocab.add(word)
    V = len(vocab)

    for label in class_counts:
        # log prior
        scores[label] = math.log(class_counts[label] / sum(class_counts.values()))
        # log likelihood
        for word in words:
            count_w = word_counts[label].get(word, 0)
            scores[label] += math.log((count_w + 1) / (total_words[label] + V))

    # Chuyển log-prob -> xác suất chuẩn hóa 0-1
    max_log = max(scores.values())
    exp_scores = {label: math.exp(scores[label] - max_log) for label in scores}
    sum_exp = sum(exp_scores.values())
    probs = {label: exp_scores[label] / sum_exp for label in exp_scores}

    if verbose:
        for label, p in probs.items():
            print(f"Xác suất {label}: {p:.4f}")

    pred_label = max(probs, key=probs.get)
    return pred_label, probs


In [12]:
# Cell 8: Nhập Subject + Body, in form chuẩn, lưu vào CSV
import datetime
import os

# Nhập email
subj_input = input("\nNhập Subject (tiêu đề): ")
body_input = input("Nhập Body (nội dung): ")

email_text = subj_input + " " + body_input

# Dự đoán
pred_label, probs = predict_proba(email_text, verbose=False)
pred_prob = probs[pred_label]

# In kết quả
print("\n--- KẾT QUẢ DỰ ĐOÁN ---")
print(f"Subject:\n{subj_input}")
print(f"Body:\n{body_input}\n")
print(f"Dự đoán: {pred_label}")
print(f"Tỉ lệ (%): {pred_prob * 100:.2f}%")

# Thêm thời gian
time_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Chuẩn bị DataFrame lưu
new_row = pd.DataFrame([{
    "subject": subj_input,
    "body": body_input,
    "pred": pred_label,
    "probability (%)": pred_prob * 100,
    "time": time_now
}])

# Nếu file CSV đã tồn tại, append; nếu chưa, tạo mới
csv_file = "filter_history.csv"
if os.path.exists(csv_file):
    new_row.to_csv(csv_file, mode='a', index=False, header=False, encoding='utf-8')
else:
    new_row.to_csv(csv_file, index=False, encoding='utf-8')

print(f"\nĐã lưu dự đoán vào {csv_file}")



--- KẾT QUẢ DỰ ĐOÁN ---
Subject:
[TB] Chương trình Khách hàng thân thiết VinID xin trân trọng thông báo
Body:
Kính gửi Quý Khách hàng,​

Chương trình Khách hàng thân thiết VinID xin trân trọng thông báo:

​

Từ tháng 08/2024, VinID chính thức chuyển mình trở thành OneU - Nền tảng khách hàng thân thiết và thanh toán tiện lợi lớn nhất Việt Nam, nhằm đem đến cho Quý Khách hàng nhiều đặc quyền vượt trội hơn.

1,332 điểm tích lũy của Quý khách sẽ bị thu hồi vào ngày 30/09/2025 23:59:59 do quá hạn sử dụng.

Để tránh lãng phí số điểm, Quý khách vui lòng cập nhật hoặc tải ứng dụng OneU để dùng điểm. Quý khách có thể sử dụng dịch vụ tiêu điểm VinID tại hệ thống Vinpearl, Vinmec, Vinschool, WinMart/WinMart+/WiN và các công ty thành viên. Ngoài ra, điểm VinID có thể được dùng để đổi ưu đãi tại tính năng “Voucher", đổi thẻ điện thoại, mua vé VinWonders trên ứng dụng OneU.

​

Để tìm hiểu chi tiết về điều khoản và điều kiện chương trình Khách hàng thân thiết VinID, Quý khách vui lòng xem tại đây

